In [ ]:
################################################################################                                                                                                                                                              
################################################################################
#
# FILE: basketball-download.py
#
# BY: Dmitry Sedov 
#
# CREATED: Tue Apr 14 2020
#
# DESC: This code downloads data about basketball teams / games from 
#       https://www.basketball-reference.com
#
# EXEC:
#      
################################################################################
################################################################################

In [ ]:
################################ Libraries #####################################

import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os

################################################################################

In [ ]:
################################ Constants #####################################

main_url = 'https://www.basketball-reference.com'
teams_url = 'https://www.basketball-reference.com/teams/'
wiki_url = 'https://en.wikipedia.org/wiki/National_Basketball_Association'
teams_output_folder = '/Users/muser/dfolder/Research/stadiums/data/basketball/teams/'
games_output_folder = '/Users/muser/dfolder/Research/stadiums/data/basketball/games/'

################################################################################

In [ ]:
######################### Get list of all teams ################################

teams_page = requests.get(teams_url).text
teams_soup = BeautifulSoup(teams_page)
active_teams_table = teams_soup.find('table', {'id': 'teams_active'})
head = str(active_teams_table.find('thead'))
rows = '\n'.join(str(x) for x in active_teams_table.find_all('tr', {'class': 'full_table'}))
active_teams_table = '<table>\n' + head + '\n<tbody>\n' + rows + '\n</tbody>\n' + '</table>'
active_teams_table = pd.read_html(active_teams_table)
active_teams_table = active_teams_table[0]
active_teams_table.rename(columns = {'Franchise': 'name'}, inplace = True)

################################################################################

In [ ]:
active_teams_table

In [ ]:
######################### Get links of all teams ###############################

active_teams_links = teams_soup.find('table', {'id': 'teams_active'})
temp = ({'link': tag['href'], 
         'name': tag.text} for tag in active_teams_links.find_all('a', 
                                                            attrs = {'class': None},
                                                            href = True))
active_teams_links = pd.DataFrame(temp)

################################################################################

In [ ]:
####################### Merge for a teams dataset ##############################

basketball_teams = pd.merge(active_teams_table, 
                            active_teams_links, 
                            how = 'outer',
                            on = 'name', 
                            validate = 'one_to_one')

# Change links for some teams
basketball_teams.loc[basketball_teams['name'] == 'Brooklyn Nets', 'link'] = '/teams/BRK/'
basketball_teams.loc[basketball_teams['name'] == 'Charlotte Hornets', 'link'] = '/teams/CHO/'
basketball_teams.loc[basketball_teams['name'] == 'New Orleans Pelicans', 'link'] = '/teams/NOP/'

################################################################################

In [ ]:
basketball_teams

In [ ]:
####################### Function to get the stadium ############################

pattern = re.compile(r'Arena')

def get_stadium(link, year):
    print(link)
    team_year_url = main_url + link + str(year) + '.html'
    team_year_soup = BeautifulSoup(requests.get(team_year_url).text, 
                                   'html.parser')
    stadium = team_year_soup.find('strong',
                                  text = pattern).next_sibling.strip()
    return stadium
    
################################################################################

In [ ]:
basketball_teams['stadium_2017'] = basketball_teams.apply(lambda row: get_stadium(row['link'], 2017),
                                                          axis = 1)

In [ ]:
basketball_teams['stadium_2018'] = basketball_teams.apply(lambda row: get_stadium(row['link'], 2018),
                                                          axis = 1)

In [ ]:
basketball_teams['stadium_2019'] = basketball_teams.apply(lambda row: get_stadium(row['link'], 2019),
                                                          axis = 1)

In [ ]:
basketball_teams['stadium_2020'] = basketball_teams.apply(lambda row: get_stadium(row['link'], 2020),
                                                          axis = 1)

In [ ]:
basketball_teams['code'] = basketball_teams['link'].apply(lambda x: x.split('/')[2])

In [ ]:
basketball_teams

In [ ]:
basketball_teams.to_csv(os.path.join(teams_output_folder, 'basketball_teams.csv'), 
                        index = False)

In [ ]:
######################## Function to get the games ############################

def get_games(link, year):
    team_code = link.split('/')[2]
    print(team_code)
    schedule_year_url = main_url + link + str(year) + '_games.html'
    schedule_tables = pd.read_html(schedule_year_url)
    season = schedule_tables[0]
    # Filter non-games rows out
    mask = season.iloc[:,0].apply(lambda x: x.isdigit())
    season_schedule_table = season.loc[mask].copy()
    season_schedule_table['team_code'] = team_code
    season_schedule_table.to_csv(os.path.join(games_output_folder,
                                              str(year),
                                              f'{team_code}_season_games.csv'),
                                 index = False)
    try:
        playoff = schedule_tables[1]
    except IndexError:
        print(f'No playoffs for {team_code}.')
        return None
    try:
        mask = playoff.iloc[:,0].apply(lambda x: x.isdigit())
        playoff_schedule_table = playoff.loc[mask].copy()
    except AttributeError:
        playoff_schedule_table = playoff
    playoff_schedule_table['team_code'] = team_code
    playoff_schedule_table.to_csv(os.path.join(games_output_folder,
                                               str(year),
                                               f'{team_code}_playoff_games.csv'), 
                                  index = False)
    return None
    
################################################################################

In [ ]:
basketball_teams.apply(lambda row: get_games(row['link'], 2017), axis = 1)

In [ ]:
basketball_teams.apply(lambda row: get_games(row['link'], 2018), axis = 1)

In [ ]:
basketball_teams.apply(lambda row: get_games(row['link'], 2019), axis = 1)

In [ ]:
basketball_teams.apply(lambda row: get_games(row['link'], 2020), axis = 1)

In [ ]:
# Download the list of the teams locations from Wikipedia
wiki_tables = pd.read_html(wiki_url, attrs = {'class': 'wikitable'})

In [ ]:
teams_locations = wiki_tables[0].droplevel(1, axis = 1)
mask = (teams_locations['Capacity'] != 'Western Conference')
teams_locations = teams_locations[mask].sort_values('Team').reset_index(drop = True)

In [ ]:
teams_locations.to_csv(os.path.join(teams_output_folder, 
                                    'basketball_locations.csv'), 
                       index = False)

In [ ]:
basketball_teams = pd.read_csv(os.path.join(teams_output_folder, 'basketball_teams.csv'))

In [ ]:
pd.merge(basketball_teams, 
         teams_locations, 
         how = 'left', 
         left_on = 'name',
         right_on = 'Team', 
         validate = 'one_to_one').shape